In [15]:
import numpy as np                                                                                    # Algèbre linéaire
import torch                                                                                          # pytorch
import transformers                                                                                   # huggingface transformers
from sklearn.model_selection import train_test_split                                                  # division train/test
from transformers import AutoTokenizer                                                                # tokenizer
import pandas as pd                                                                                   # traitement de données
from datasets import load_dataset                                                                     # chargement de dataset
import glob                                                                                           # glob pour les fichiers
import os                                                                                             # os pour les fichiers
import re                                                                                             # regex
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments,AutoModelWithLMHead   # modèle T5

In [9]:
def read_data(path):                                                              # fonction pour lire les fichiers
    data = []
    for topic in os.listdir(path):
        for file in os.listdir(path + "/" + topic):                               # parcourir les fichiers par topic
            with open(path + "/" + topic + "/" + file) as f:
                data.append(f.read())
    return data

original_text = read_data("files/BBC News Summary/Summaries")                      # lire le texte original
summary_text = read_data("files/BBC News Summary/News Articles")                   # lire le texte résumé

df = pd.DataFrame({'original':original_text,'summary':summary_text})               # créer un dataframe avec les deux colonnes

df.to_csv('files/summary.csv', index=False)                                        # sauvegarder le dataframe en csv

In [10]:
dataset = load_dataset('csv', data_files='files/summary.csv', split='train')       # charger le dataset
dataset = dataset.train_test_split(test_size=0.1)                                  # diviser le dataset en train/test
train_dataset = dataset['train']                                                   # train dataset
val_dataset = dataset['test']                                                      # test dataset

Using custom data configuration default-49933766bd4604e6


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 285.17it/s]
Generating train split: 0 examples [00:00, ? examples/s]C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\datasets\download\streaming_download_manager.py:714: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)
                                                                 

Dataset csv downloaded and prepared to C:/Users/moham/.cache/huggingface/datasets/csv/default-49933766bd4604e6/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


In [11]:
tokenizer = AutoTokenizer.from_pretrained('t5-base')                                 # charger le tokenizer

def tokenize(batch):                                                                 # fonction pour tokeniser les données
    tokenized_input = tokenizer(batch['original'],                                   # tokenizer le texte original
                                padding='max_length',                                # ajouter du padding pour avoir des inputs de même taille
                                truncation=True,                                     # tronquer les inputs trop longs
                                max_length=512)                                      # taille max de l'input (512 pour T5)
    tokenized_label = tokenizer(batch['summary'], 
                                padding='max_length', 
                                truncation=True, 
                                max_length=159)

    tokenized_input['labels'] = tokenized_label['input_ids']                         # ajouter les labels aux inputs

    return tokenized_input                                                           # retourner les inputs tokenisés

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=512)            # tokenizer le dataset d'entrainement
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))   # tokenizer le dataset de test

train_dataset.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels']) # mettre le format du dataset d'entrainement en numpy
val_dataset.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])   # mettre le format du dataset de test en numpy

C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
100%|██████████| 1/1 [00:00<00:00,  1.03ba/s]


In [ ]:
model = T5ForConditionalGeneration.from_pretrained('t5-base')                         # charger le modèle T5

output_dir = './output_dir'                                                           # dossier de sortie

training_args = TrainingArguments(                                                    # définition des paramètres d'entrainement
    output_dir=output_dir,                                                            # dossier de sortie
    num_train_epochs=1,                                                               # nombre d'époques d'entraînement, nous le définissons sur 1 car nous utilisons wandb 
                                                                                      # pour suivre le processus d'entraînement
                                                                                      # et on peut arrêter l'entraînement quand on veut, 
                                                                                      # en plus chaque époque prend beaucoup de temps
    per_device_train_batch_size=8,                                                    # taille du batch d'entraînement, nous le définissons sur 8 car on a un GPU avec 128MB de vRAM
    per_device_eval_batch_size=8,                                                     # 
    eval_accumulation_steps=1,                                                        # nombre d'étapes d'évaluation à conserver dans le GPU (plus il est élevé, plus la vRAM utilisée)
    prediction_loss_only=True,                                                        # si on a besoin de co calculer uniquement la perte et pas d'autres mesures, 
                                                                                      # le régler sur vrai utilisera moins de RAM
    learning_rate=0.1,                                                                # taux d'apprentissage (plus il est élevé, plus l'entraînement est rapide, 
                                                                                      # mais il peut conduire à la divergence)
    evaluation_strategy='steps',                                                      # comment évaluer le modèle, ici on évalue le modèle à chaque 1000 étapes
    save_steps=1000,                                                                  # sauvegarder le modèle à chaque 1000 étapes
    save_total_limit=1,                                                               # nombre maximal de modèles à sauvegarder
    remove_unused_columns=True,                                                       # supprimer les colonnes non utilisées du dataset
    run_name='run_name',                                                              # nom de l'entraînement (pour wandb)
    logging_steps=1000,                                                               # enregistrer les logs à chaque 1000 étapes
    eval_steps=1000,                                                                  # évaluer le modèle à chaque 1000 étapes
    logging_first_step=False,                                                         # ne pas enregistrer les logs pour la première étape
    load_best_model_at_end=True,                                                      # charger le meilleur modèle à la fin de l'entraînement
    metric_for_best_model="loss",                                                     # métrique à utiliser pour déterminer le meilleur modèle (ici la perte)
    greater_is_better=False                                                           # si la métrique est meilleure quand elle est plus grande ou plus petite
)

trainer = Trainer(                                                                    # définition du trainer
    model=model,                                                                      # modèle à entrainer
    args=training_args,                                                               # paramètres d'entrainement
    train_dataset=train_dataset,                                                      # dataset d'entrainement    
    eval_dataset=val_dataset                                                          # dataset d'évaluation
)

trainer.train()                                                                       

In [7]:
trainer.save_model(output_dir + '/model_')                                            # sauvegarder le modèle pour le réutiliser utérieurement

Saving model checkpoint to ./output_dir/model_
Configuration saved in ./output_dir/model_\config.json
Model weights saved in ./output_dir/model_\pytorch_model.bin


In [10]:
from transformers import pipeline

summarizer = pipeline('summarization',                                                # pipeline de résumé automatique 
                        model=output_dir + '/model_',                                 # chemin du modèle
                        tokenizer=tokenizer,                                          # tokenizer
                        framework='pt')                                               # framework utilisé (pytorch) 

loading configuration file ./output_dir/model_\config.json
Model config T5Config {
  "_name_or_path": "./output_dir/model_",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
   

# <p style='color:red;'>Notice</p> 
#### Il y a une grosse limite ici, l'entraînement se fait sur un seul GPU, donc ça prend beaucoup de temps pour entraîner le modèle, j'ai essayé de l'entraîner sur un Google Colab Pro, mais c'était encore trop lent, j'ai donc décidé d'utiliser wandb pour suivre le processus d'entraînement et l'arrêter quand je le souhaite, puis j'enregistre le modèle et l'utilise pour générer des résumés.

#### Même si j'ai entraîné le modèle pendant 1 époque, il m'a fallu plus de 6 heures pour l'entraîner, j'ai donc décidé d'utiliser un modèle pré-entraîné, que je vous montrerai dans la section suivante.


## Génération de résumés à l'aide du modèle t5-small

In [26]:
def generate_summary(text):                                                              # fonction pour générer le résumé

    model = AutoModelWithLMHead.from_pretrained("t5-small")                              # charger le modèle
    tokenizer = AutoTokenizer.from_pretrained("t5-small")                                # charger le tokenizer

    inputs = tokenizer.encode("summarize: " + text,                                      # encode le texte à résumer
                                return_tensors="pt",                                     # type de tenseur de sortie (ici pytorch)
                                max_length=512,                                          # taille maximale de l'entrée (512 pour T5)
                                truncation=True)                                         # tronquer le texte si sa taille est supérieure à la taille maximale de l'entrée

    outputs = model.generate(inputs,                                                     # générer le résumé
                                max_length=250,                                          # taille maximale de la sortie (ici 250) 
                                min_length=80,                                           # taille minimale de la sortie (ici 80)
                                length_penalty=2.0,                                      # pénalité de longueur (plus elle est élevée, plus le modèle génère un résumé plus long)
                                num_beams=4,                                             # numéro de faisceaux (plus il est élevé, plus le modèle génère un résumé plus long) 
                                early_stopping=True)                                     # si on veut arrêter l'entraînement quand on atteint la taille maximale de la sortie 
    return tokenizer.decode(outputs[0])                                                  # retourner le résumé


def generate_summary_for_text(text):                                                     # fonction pour générer le résumé pour un texte donné

    # text = re.sub(r'\d+', '', text)                                                    # supprimer les chiffres du texte (si on veut)
    text = re.sub(' +', ' ', text)                                                       # supprimer les espaces multiples
    text = re.sub('\n+', ' ', text)                                                      # supprimer les sauts de ligne multiples
    text = re.sub('\t+', ' ', text)                                                      # supprimer les tabulations multiples

    summary = generate_summary(text)                                                     # générer le résumé du texte             
    return summary                                                                       # retourner le résumé généré

In [27]:
summary_ = generate_summary_for_text(df['original'][5])                                  # générer le résumé pour le texte d'index 5

print('Predicted summary :\n',summary_)
print(" ")
print('Original summary :\n',df['summary'][5])

def count_words(text):
    return len(text.split())

print('Predicted summary word count : ',count_words(summary_))
print('Original summary word count : ',count_words(df['summary'][5]))

Predicted summary :
 <pad> a common technical definition of a recession is two successive quarters of negative growth. on an annual basis, the data suggests annual growth of just 0.2%, suggesting a much more hesitant recovery than had previously been thought. japan's economy teetered on the brink of a technical recession in the three months to September, figures show.</s>
 
Original summary :
 Japan narrowly escapes recession

Japan's economy teetered on the brink of a technical recession in the three months to September, figures show.

Revised figures indicated growth of just 0.1% - and a similar-sized contraction in the previous quarter. On an annual basis, the data suggests annual growth of just 0.2%, suggesting a much more hesitant recovery than had previously been thought. A common technical definition of a recession is two successive quarters of negative growth.

The government was keen to play down the worrying implications of the data. "I maintain the view that Japan's economy 